# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,42.46,70,40,19.57,AR,1728172138
1,1,edinburgh of the seven seas,-37.0676,-12.3116,54.01,89,100,23.22,SH,1728172139
2,2,petropavlovsk-kamchatsky,53.0452,158.6483,48.79,43,0,2.24,RU,1728172030
3,3,san antonio oeste,-40.7319,-64.9477,57.69,45,28,11.79,AR,1728172145
4,4,nagqu,31.5000,92.0000,35.71,82,82,1.50,CN,1728172147


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
#%%capture --no-display
# Configure the map plot_1
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='humidity', 
    tiles='OSM', 
    title='City Humidity Map',
    color='City',
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter for cities with ideal conditions
#A max temperature lower than 90 degrees but higher than 60
ideal_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 60) & 
    (city_data_df["Max Temp"] <= 90) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop results with null values
ideal_city_df = ideal_city_df.dropna()

# Display the resulting DataFrame
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,bethel,41.3712,-73.4140,61.90,69,0,1.01,US,1728172209
43,43,katsuren-haebaru,26.3369,127.8719,85.08,82,0,0.00,JP,1728172220
75,75,stephenville,32.2207,-98.2023,82.53,42,0,0.00,US,1728172291
155,155,sirsa,29.5333,75.0167,80.01,40,0,3.24,IN,1728172421
156,156,crane,31.3974,-102.3501,83.53,25,0,1.16,US,1728172422
172,172,matoury,4.8483,-52.3317,80.56,83,0,3.44,GF,1728172445
179,179,concepcion,-36.8270,-73.0498,60.60,67,0,0.00,CL,1728172458
280,280,remire-montjoly,4.9167,-52.2667,80.64,83,0,3.44,GF,1728172643
289,289,mut,36.6439,33.4389,70.84,47,0,0.47,TR,1728172659
324,324,hamilton,39.1834,-84.5333,73.22,66,0,1.99,US,1728172726


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
37,bethel,US,41.3712,-73.4140,69,
43,katsuren-haebaru,JP,26.3369,127.8719,82,
75,stephenville,US,32.2207,-98.2023,42,
155,sirsa,IN,29.5333,75.0167,40,
156,crane,US,31.3974,-102.3501,25,
172,matoury,GF,4.8483,-52.3317,83,
179,concepcion,CL,-36.8270,-73.0498,67,
280,remire-montjoly,GF,4.9167,-52.2667,83,
289,mut,TR,36.6439,33.4389,47,
324,hamilton,US,39.1834,-84.5333,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
katsuren-haebaru - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
sirsa - nearest hotel: Hotel Gumber
crane - nearest hotel: No hotel found
matoury - nearest hotel: La Marmotte
concepcion - nearest hotel: Hotel One (ex-Biobio)
remire-montjoly - nearest hotel: Complexe Belova
mut - nearest hotel: Deveci Otel
hamilton - nearest hotel: North Vista Manor
metlili chaamba - nearest hotel: No hotel found
jijel - nearest hotel: Hôtel Louiza
byron bay - nearest hotel: Lord Byron Resort
westport - nearest hotel: Norwalk Inn and Conference Center
jackson - nearest hotel: Brookside
tomah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
37,bethel,US,41.3712,-73.4140,69,Hampton Inn Danbury
43,katsuren-haebaru,JP,26.3369,127.8719,82,No hotel found
75,stephenville,US,32.2207,-98.2023,42,No hotel found
155,sirsa,IN,29.5333,75.0167,40,Hotel Gumber
156,crane,US,31.3974,-102.3501,25,No hotel found
172,matoury,GF,4.8483,-52.3317,83,La Marmotte
179,concepcion,CL,-36.8270,-73.0498,67,Hotel One (ex-Biobio)
280,remire-montjoly,GF,4.9167,-52.2667,83,Complexe Belova
289,mut,TR,36.6439,33.4389,47,Deveci Otel
324,hamilton,US,39.1834,-84.5333,66,North Vista Manor


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
#%%capture --no-display
hotel_df = hotel_df.dropna()
# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles="OSM",
    color="City",
    scale=0.5,
    size = "Humidity",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)